In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
import spacy as sp
# Industrial-Strength Natural Language Processing
is_using_gpu = sp.prefer_gpu()
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
dataset=pd.read_csv("/content/goodreads_books_small.csv")
dataset


,id,title,link,series,cover_link,author,author_link,rating_count,review_count,average_rating,five_star_ratings,four_star_ratings,three_star_ratings,two_star_ratings,one_star_ratings,number_of_pages,date_published,publisher,original_title,genre_and_votes,isbn,isbn13,asin,settings,characters,awards,amazon_redirect_link,worldcat_redirect_link,recommended_books,books_in_series,description,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47
0,630104,Inner Circle,https://www.goodreads.com//book/show/630104.In...,(Private #5),https://i.gr-assets.com/images/S/compressed.ph...,"Kate Brian, Julian Peploe",https://www.goodreads.com/author/show/94091.Ka...,7597,196,4.03,3045.0,2323.0,1748,389,92,220.0,January 1st 2007,Simon Schuster Books for Young Readers,Inner Circle,"Young Adult 161, Mystery 45, Romance 32",1416950419,9781416950417,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"726458, 726458, 1537534, 3047848, 1651302, 304...","381489, 381501, 352428, 630103, 1783281, 17832...",Reed Brennan arrived at Easton Academy expecti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9487,A Time to Embrace,https://www.goodreads.com//book/show/9487.A_Ti...,(Timeless Love #2),https://i.gr-assets.com/images/S/compressed.ph...,Karen Kingsbury,https://www.goodreads.com/author/show/3159984....,4179,177,4.35,2255.0,1290.0,518,93,23,400.0,October 29th 2006,Thomas Nelson,NaN,"Christian Fiction 114, Christian 45, Fiction 3...",1595542329,9781595542328,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"127352, 127352, 40642197, 127353, 127354, 3891...",115036,"Ideje az Ã¶lelÃ©snek TÃ¶rtÃ©net a remÃ©nyrÅl,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6050894,Take Two,https://www.goodreads.com//book/show/6050894-t...,(Above the Line #2),https://i.gr-assets.com/images/S/compressed.ph...,Karen Kingsbury,https://www.goodreads.com/author/show/3159984....,6288,218,4.23,3000.0,2020.0,1041,183,44,320.0,January 1st 2009,Zondervan,Take Two,"Christian Fiction 174, Christian 81, Fiction 58",NaN,NaN,NaN,"Bloomington, Indiana(United States)",NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"706250, 706250, 666481, 11942636, 706241, 1273...","4010795, 40792877, 7306261",Filmmakers Chase Ryan and Keith Ellison have c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39030,Reliquary,https://www.goodreads.com//book/show/39030.Rel...,(Pendergast #2),https://i.gr-assets.com/images/S/compressed.ph...,"Douglas Preston, Lincoln Child",https://www.goodreads.com/author/show/12577.Do...,38382,1424,4.01,12711.0,15407.0,8511,1429,324,464.0,1997,Tor Books,Reliquary,"Thriller 626, Mystery 493, Horror 432, Fiction...",765354950,9780765354952,NaN,"New York State(United States) New York City, N...","Aloysius X.L. Pendergast, Margo Green, William...",NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"39026, 39026, 11007668, 3986318, 51887330, 194...","67035, 39031, 39033, 136637, 136638, 30068, 39...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,998,The Millionaire Next Door: The Surprising Secr...,https://www.goodreads.com//book/show/998.The_M...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,"Thomas J. Stanley, William D. Danko",https://www.goodreads.com/author/show/659.Thom...,72168,3217,4.04,27594.0,25219.0,14855,3414,1086,258.0,October 28th 1995,Gallery Books,The Millionaire Next Door: The Surprising Secr...,"Economics-Finance 1162, Nonfiction 910, Busine...",671015206,9780671015206,NaN,NaN,NaN,Independent Publisher Book Award (IPPY) Nomine...,https://www.goodreads.com//book_link/fo

In [ ]:
data1=dataset[['title','description']]
data1

,title,description
0,Inner Circle,Reed Brennan arrived at Easton Academy expecti...
1,A Time to Embrace,"Ideje az Ã¶lelÃ©snek TÃ¶rtÃ©net a remÃ©nyrÅl,..."
2,Take Two,Filmmakers Chase Ryan and Keith Ellison have c...
3,Reliquary,NaN
4,The Millionaire Next Door: The Surprising Secr...,The incredible national bestseller that is cha...
...,...,...
9995,Anne McCaffrey Freedom Collection: Freedom's L...,"Freedom's Landing - read by Susie Breck, direc..."
9996,"Break the Bodies, Haunt the Bones",Jane is haunted. Since she w Jane is haunted. ...
9997,Ð¡Ð°Ð¼Ð¾Ð´Ð¸Ð²Ð°,Ð¢ÐµÐ¾Ð´Ð¾ÑÐ° Ð¡Ð»Ð°Ð²Ð¸Ð½ Ð¿ÑÐ¸ÑÑÐ¸Ð³Ð° Ð...
9998,Brief Answers to the Big Questions,Stephen Hawking was the most renowned scientis...


In [ ]:
print(data1['title'][0])

Inner Circle


In [ ]:
data1['description'] =data1['description'].astype(str)
# we take the description 
data1['len']=data1['description'].apply(lambda x:len(x.split(' ')))
bookdf=data1[data1['len']>=50]
# bookdf.shape

import en_core_web_sm
nlp = en_core_web_sm.load()
import re
def clean_text(doc):
    #Clean the document. Remove pronouns, stopwords, lemmatize the words and lowercase them
    doc = nlp(doc)
    tokens = []
    exclusion_list = ["nan"]
    for token in doc:
        if token.is_stop or token.is_punct or token.text.isnumeric() or (token.text.isalnum()==False) or token.text in exclusion_list :
            continue
        token = str(token.lemma_.lower().strip())
        tokens.append(token)
    return " ".join(tokens) 
bookdf['cleaned_description']=bookdf.apply(lambda row: clean_text(row["description"]) ,axis=1)
bookdf.reset_index(inplace=True)

# newdf=bookdf[['title','description','cleaned_description']]
newdf=bookdf[['title','cleaned_description']]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [ ]:
newdf.to_csv('data.csv')
!cp data.csv "drive/My Drive/"

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
newdf['title_id']=le.fit_transform(newdf['title'])
contentdf=newdf[['title_id','title','description','cleaned_description']]
contentdf=contentdf.set_index('title_id')
contentdf.reset_index(inplace=True)
#contentdf.drop(['index'],axis=1,inplace=True)
# contentdf

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=3,ngram_range=(1, 3), stop_words='english')
tfidf_description = tfidf.fit_transform((contentdf["cleaned_description"])) 
print("Each of the %d text is represented by %d features (TF-IDF score of unigrams and bigrams)" %(tfidf_description.shape)) 
# Finding cosine similarity between vectors 
from sklearn.metrics.pairwise import cosine_similarity 
cos_sim = linear_kernel(tfidf_description, tfidf_description) 


indices = pd.Series(contentdf.index) 
def recommendations(title, cosine_sim = cos_sim): 
    recommended_book = [] 
    index = indices[indices == title].index[0]
    similarity_scores = pd.Series(cosine_sim[index]).sort_values(ascending = False) 
    top_10_books = list(similarity_scores.iloc[1:11].index)
    for i in top_10_books: 
        recommended_book.append(list(contentdf.index)[i]) 
    top_5_books=[]
    for i in recommended_book:
        if i not in top_5_books and i!=title:
            top_5_books.append(i)
    recommendbook=[]
    for i in top_5_books:
        recommendbook.append(contentdf['title'][i])
    return recommendbook


book="Inner Circle"
print("Enter a book you liked: ",book)
a=contentdf.loc[contentdf['title']==book]
id=a.index[0]
r=recommendations(id)
print("*****Here are a few recommendations for you*****")
for i,j in zip(r,range(1,len(r)+1)):
    print(j,".",i)



In [ ]:
newdf

,title,cleaned_description
0,Inner Circle,reed brennan arrive easton academy expect find...
1,A Time to Embrace,ideje az snek net remã ã s az ã let karen king...
2,Take Two,filmmaker chase ryan keith ellison complete fe...
3,Black Sheep,high spirited intelligence good look abigail w...
4,Sylvester,news go sylvester rayne elegant impeccable duk...
...,...,...
8630,Anne McCaffrey Freedom Collection: Freedom's L...,freedom landing read susie breck direct ruth b...
8631,"Break the Bodies, Haunt the Bones",jane haunt w jane haunt child carry ghost girl...
8632,Ð¡Ð°Ð¼Ð¾Ð´Ð¸Ð²Ð°,ð² ð½ð ð¼ð ð² ð½ð ð² ð ð ð½ð ð½ðµ ð ðµ ð² ð½ð ...
8633,Brief Answers to the Big Questions,stephen hawking renowned scientist einstein kn...


In [ ]:
print(tfidf_description)

  (0, 32006)	0.07404754449820569
  (0, 806)	0.1000501407472895
  (0, 24392)	0.1026908865910505
  (0, 8343)	0.1026908865910505
  (0, 5145)	0.1026908865910505
  (0, 23480)	0.08704884262274759
  (0, 29199)	0.1026908865910505
  (0, 29720)	0.14264993385514707
  (0, 23574)	0.1371509886117118
  (0, 3162)	0.1693996137208989
  (0, 19119)	0.17387078510083825
  (0, 8874)	0.17387078510083825
  (0, 16845)	0.1693996137208989
  (0, 19344)	0.1576220163037112
  (0, 23518)	0.10522555111173973
  (0, 6875)	0.1397124111073649
  (0, 26810)	0.14264993385514707
  (0, 9630)	0.1657464007022747
  (0, 28130)	0.1693996137208989
  (0, 31991)	0.04840089362348275
  (0, 21642)	0.0500805007130305
  (0, 802)	0.05347345166009284
  (0, 37207)	0.0558880561334148
  (0, 24307)	0.0329266574038184
  (0, 36801)	0.06263902549554064
  :	:
  (8634, 26163)	0.22701208504241574
  (8634, 9759)	0.16506539503769518
  (8634, 6390)	0.21709687536055267
  (8634, 2771)	0.2026404975221934
  (8634, 28215)	0.2701750249866549
  (8634, 29458)	0.2

In [ ]:
print(cos_sim)

[[1.         0.         0.00325611 ... 0.         0.01346822 0.01267352]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.00325611 0.         1.         ... 0.         0.01363674 0.00507293]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.01346822 0.         0.01363674 ... 0.         1.         0.00190927]
 [0.01267352 0.         0.00507293 ... 0.         0.00190927 1.        ]]


In [ ]:
contentdf.to_csv('content.csv', index = False)

In [ ]:
import pickle
string = pickle.dumps(cos_sim)
with open("a_file.txt", "wb") as f:
    f.write(string)